In [3]:

import csv
import re

with open("data.txt", 'r') as file:
    T = [list(map(int,rec)) for rec in csv.reader(file, delimiter=',')]



In [4]:
with open("mis.txt", 'r') as file2:                                                 #doesnt work if mis.txt has \n in the end.
    a=file2.read()
    
d= {}
for x in a.split("\n"):
    (k,v) = x.split("=")
    d[k.strip()] = float(v.strip())

sdc = d['SDC']

sorted_mis = {k: v for k, v in sorted(d.items(), key=lambda item: item[1])}    #M sorted items according to MIS values.

print(sorted_mis)

{'MIS(1)': 0.02, 'MIS(2)': 0.04, 'MIS(rest)': 0.01, 'SDC': 0.003}
{'MIS(rest)': 0.01, 'MIS(1)': 0.02, 'MIS(2)': 0.04}


In [5]:
support_count = {}
unique_items = []
for transaction in T:
    for item in transaction:
        support_count[item] = 0

I = []
for key in support_count:
    I.append(key)

temp_I = I
for key in sorted_mis:                                                         #if MIS.txt has an item that is not used
                                                                               #in the transactions file exception will 
                                                                               #be thrown
    if key == 'MIS(rest)':
        continue
    item = int(re.findall(r'\d+', key)[0])
    temp_I.remove(item)
    
    
for item in temp_I:
    sorted_mis['MIS('+str(item)+')'] = sorted_mis['MIS(rest)']
sorted_mis.pop('MIS(rest)', None)

M_ = {k: v for k, v in sorted(sorted_mis.items(), key=lambda item: item[1])}   #consists of sorted values

print(M_)
M = {}
for key in M_:
    M[int(re.findall(r'\d+', key)[0])] = M_[key]                               #conversion of string keys to int
print(M)
for key in support_count:
    for transaction in T:
        if key in transaction:
            support_count[key] += 1
            continue
M_keys = []    
for key in M:
    M_keys.append(key)
""" creating L from sorted lsit of items in the order of ascending mis values.""" 
L = []
L.append(M_keys[0])
for item in M_keys[1:]:
    if support_count[item]/len(T) >= M[M_keys[0]]:
        L.append(item)
        
""" creating F1 frequent itemset """
F1 = []
for l in L:
    if support_count[l]/len(T) >= M[l]:
        F1.append(l)
        
""" Variables we have """
print("M is a dictionary that consists of items as keys and MIS as values:",M)
print("M_keys consists of all items in M sorted in the order of ascending MIS values:",M_keys)
print("L consists of items satisfying the algorithm",L)
print("F1 is the frequent itemset",F1)

""" Candidate Generation Function """
#working

{'MIS(10)': 0.01, 'MIS(50)': 0.01, 'MIS(30)': 0.01, 'MIS(40)': 0.01, 'MIS(4)': 0.01, 'MIS(6)': 0.01, 'MIS(8)': 0.01, 'MIS(9)': 0.01, 'MIS(3)': 0.01, 'MIS(5)': 0.01, 'MIS(7)': 0.01, 'MIS(20)': 0.01, 'MIS(1)': 0.02, 'MIS(2)': 0.04}
{10: 0.01, 50: 0.01, 30: 0.01, 40: 0.01, 4: 0.01, 6: 0.01, 8: 0.01, 9: 0.01, 3: 0.01, 5: 0.01, 7: 0.01, 20: 0.01, 1: 0.02, 2: 0.04}
M is a dictionary that consists of items as keys and MIS as values: {10: 0.01, 50: 0.01, 30: 0.01, 40: 0.01, 4: 0.01, 6: 0.01, 8: 0.01, 9: 0.01, 3: 0.01, 5: 0.01, 7: 0.01, 20: 0.01, 1: 0.02, 2: 0.04}
M_keys consists of all items in M sorted in the order of ascending MIS values: [10, 50, 30, 40, 4, 6, 8, 9, 3, 5, 7, 20, 1, 2]
L consists of items satisfying the algorithm [10, 50, 30, 40, 4, 6, 8, 9, 3, 5, 7, 20, 1, 2]
F1 is the frequent itemset [10, 50, 30, 40, 4, 6, 8, 9, 3, 5, 7, 20, 1, 2]


' Candidate Generation Function '

In [39]:
F={}
F[1]=F1
k=2

F[2]=[(40,4),(50,3),(20,5),(40,3)] # dummy value
while(k-1) in F:
    if k==2:
        ## Do k=2
        pass
    else: # candidate generation 
        F_k_1=F[k-1]
        C=[]
        for i in range(len(F_k_1)):
            for j in range(i+1, len(F_k_1)): 
                f1 = list(F_k_1[i])[:k-2]
                f2 = list(F_k_1[j])[:k-2]
                if f1==f2:
                    ik_1=F_k_1[i][-1] 
                    ik_2=F_k_1[j][-1]
                    #print(ik_1,ik_2)
                    #print('M[ik_1]',M[ik_1])
                    #print('M[ik_2]',M[ik_2])
                    if M_keys.index(ik_1)<M_keys.index(ik_2) and (abs(M[ik_1]-M[ik_2])<=sdc): #add candidate 
                        c=list(F_k_1[i])
                        c.append(ik_2)
                        flag=False
                        
                        for window in range(k-1): #prune the candidate list 
                            s=c[window:k-1]
                            if (c[0] in s) or (M[c[1]]==M[c[0]]):
                                if s in F_k_1:
                                    flag=True
                                    break
                        
                        if not (flag):
                            C.append(set(c))
        if len(C)>0:
            F[k]=C
    k+=1
    print(F)

{1: [10, 50, 30, 40, 4, 6, 8, 9, 3, 5, 7, 20, 1, 2], 2: [(40, 4), (50, 3), (20, 5), (40, 3)]}
{1: [10, 50, 30, 40, 4, 6, 8, 9, 3, 5, 7, 20, 1, 2], 2: [(40, 4), (50, 3), (20, 5), (40, 3)], 3: [{40, 3, 4}]}
{1: [10, 50, 30, 40, 4, 6, 8, 9, 3, 5, 7, 20, 1, 2], 2: [(40, 4), (50, 3), (20, 5), (40, 3)], 3: [{40, 3, 4}]}
